# 주제

- 다중 클래스 분류

# 트랜스포머 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 120 kB 42.2 MB/s 


# 데이터 준비

- 카카오브레인 저장소에서 데이터 획득
  - NLI(Natural Language Inference) 자연어 추론
    - 두 문장(premise, hypothesis)을 입력 받아 두 관계를 
    - {entailment, contradiction, neutral}로 분류

  - STS(semantic textual similarity) 텍스트 의미적 유사성
    - 두 문장 사이의 semantic similarity(의미적 유사성)의 정도를 평가
    - 0 (dissimilar) <——-> 5 (equivalent)

  - https://github.com/kakaobrain/KorNLUDatasets
    - 카카오 제공 
    - KorNLU

  - 논문
    - https://arxiv.org/abs/2004.03289
    
  - 간편정리
    - https://hryang06.github.io/nlp/KorNLI-KorSTS/

In [ ]:
import urllib.request as req

In [ ]:
# KorNLI 데이터 획득
public_path = 'https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master'

req.urlretrieve( public_path + '/KorNLI/multinli.train.ko.tsv', filename='multinli.train.ko.tsv')
req.urlretrieve( public_path + '/KorNLI/snli_1.0_train.ko.tsv', filename='snli_1.0_train.ko.tsv')
req.urlretrieve( public_path + '/KorNLI/xnli.dev.ko.tsv', filename='xnli.dev.ko.tsv')
req.urlretrieve( public_path + '/KorNLI/xnli.test.ko.tsv', filename='xnli.test.ko.tsv')

req.urlretrieve( public_path + '/KorSTS/sts-dev.tsv', filename='sts-dev.tsv')
req.urlretrieve( public_path + '/KorSTS/sts-test.tsv', filename='sts-test.tsv')
req.urlretrieve( public_path + '/KorSTS/sts-train.tsv', filename='sts-train.tsv')

('sts-train.tsv', <http.client.HTTPMessage at 0x7f0077a63210>)

In [ ]:
# DataFrame으로 로드
import pandas as pd

In [ ]:
train_multinli = pd.read_csv('multinli.train.ko.tsv', sep='\t', quoting=3)
train_snli     = pd.read_csv('snli_1.0_train.ko.tsv', sep='\t', quoting=3)
val_xnli       = pd.read_csv('xnli.dev.ko.tsv', sep='\t', quoting=3)
test_xnli      = pd.read_csv('xnli.test.ko.tsv', sep='\t', quoting=3)

In [ ]:
train_multinli.shape, train_snli.shape, val_xnli.shape, test_xnli.shape

((392702, 3), (550152, 3), (2490, 3), (5010, 3))

In [ ]:
train_multinli.head(2)

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment


In [ ]:
train_snli.head(2)

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction


In [ ]:
# train_multinli train_snli 를 합병, 결합, 합치기->추가하기의 맥락(shape 동일하다면)
train_nli = train_multinli.append( train_snli )
# 데이터가 출처대로 합병되었다 -> 섞어라 -> 데이터가 다양하게 제공되길 원함
train_nli.shape

(942854, 3)

In [ ]:
# 샘플링 -> 전체 비율로 확대 -> 데이터가 섞기게 처리
# frac=1 : 100%  비율로 샘플링 수행 -> 섞인다 -> 사본리턴
train_nli = train_nli.sample(frac=1)
train_nli.head()
# 인덱스 값으로 섞였음을 확인할수 있다

,sentence1,sentence2,gold_label
93778,1인용 준설기로 강에서 준설작업을 하는 남성.,남자가 강에서 수영을 하고 있다.,contradiction
309351,"내가 말했듯이, 시드니로 데려가서 개회식과 횃불과 찬송가에 데려다 주면 난 괜찮을 거야.",시드니에서 열리는 개막식에 참석하면 괜찮을 거야.,entailment
131161,영수증 및 수락을 검증하기 전에 GAO 응답 지불 승인은 빠른 급여라고하는 일반적인...,빠른 임금은 일반적인 과정이다.,entailment
208830,검은 셔츠를 입은 나이든 아가씨가 군중들이 지켜보는 가운데 키친에이드 믹싱볼에 재료...,타겟 브랜드 주방용품 제품이 광고를 촬영하는 데 사용되고 있다.,contradiction
383302,빨간 재킷을 입은 소년이 눈 덮인 나무를 막대기로 때린다.,소년이 검은 재킷을 입고 있다.,contradiction


In [ ]:
# 결측제거 -> 중복제거 -> 인덱스재설정(리셋)
def getDataClean( df ):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df  

In [ ]:
train_nli = getDataClean(train_nli)
val_xnli  = getDataClean(val_xnli)
test_xnli = getDataClean(test_xnli)

# 훈련, 검증, 테스트용 데이터 준비 완료
train_nli.shape, val_xnli.shape, test_xnli.shape

((941814, 3), (2490, 3), (5010, 3))

In [ ]:
# 인덱스 정리 확인
train_nli.head()

,sentence1,sentence2,gold_label
0,1인용 준설기로 강에서 준설작업을 하는 남성.,남자가 강에서 수영을 하고 있다.,contradiction
1,"내가 말했듯이, 시드니로 데려가서 개회식과 횃불과 찬송가에 데려다 주면 난 괜찮을 거야.",시드니에서 열리는 개막식에 참석하면 괜찮을 거야.,entailment
2,영수증 및 수락을 검증하기 전에 GAO 응답 지불 승인은 빠른 급여라고하는 일반적인...,빠른 임금은 일반적인 과정이다.,entailment
3,검은 셔츠를 입은 나이든 아가씨가 군중들이 지켜보는 가운데 키친에이드 믹싱볼에 재료...,타겟 브랜드 주방용품 제품이 광고를 촬영하는 데 사용되고 있다.,contradiction
4,빨간 재킷을 입은 소년이 눈 덮인 나무를 막대기로 때린다.,소년이 검은 재킷을 입고 있다.,contradiction


# 데이터 인코딩 - 토큰화 작업

- 토크나이저

## feature 데이터 인코딩 처리

In [ ]:
from transformers import BertTokenizer, TFBertModel

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
# 토크나이저 획득
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
train_nli.columns

Index(['sentence1', 'sentence2', 'gold_label'], dtype='object')

In [ ]:
# 샘플링으로 1개씩만 확인
sent1 = train_nli.sentence1.iloc[0]
sent2 = train_nli.sentence2.iloc[0]

In [ ]:
# 문장의 최대 길이 지정(토큰화시)
max_length = 128

sample_encode = tokenizer.encode_plus( sent1, sent2, 
                                      max_length=max_length, pad_to_multiple_of=True  )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
sample_encode
# input_ids     : 토큰의 사전상의 인덱스로 표기
#                 0, 1, 2, 3, ... => 스페셜 토큰 [CLS]:문장이시작한다, [SEP]:문장이끝났다, ...
#                 [UNK]:모르는 단어다. 사전에 등록이 않되어 있다
#                 [MASK]:마스크 토큰(숨김용도), [PAD]:문장 크기 보정용
# token_type_ids: 첫번째 문장은 0, 두번째 문장은 1 : 문장 구분용
# attention_mask: 문장이면 1, 패딩이면 0

{'input_ids': [2, 21, 2179, 2048, 19769, 2015, 2200, 553, 27135, 19769, 2333, 2078, 2069, 1889, 2259, 4576, 18, 3, 3997, 2116, 553, 27135, 6379, 2069, 6159, 1513, 2062, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

- 문장1, 문장2 말뭉치를 넣어서 뒤에서 사용될 모델에 학습용 데이터로 사용되는 형태로 변환처리를 진행
- 해당 함수(encode_plus())가 문자열 상태로 처리만 되므로, 말뭉치에서 하나씩 꺼내서(pair) 인코딩 처리후 값을 분해 하여 각각 데이터를 묶어서 반환하는 함수를 만드시오
  - input_ids
  - token_type_ids
  - attention_mask

In [ ]:
# 장시간 작업시 진행율을 보여주는 기능 -> tqdm
from tqdm import tqdm

for s1, s2 in tqdm(zip(train_nli.sentence1, train_nli.sentence2), total=len(train_nli.sentence1) ):
  pass
  #print( s1, s2 )
  #break

100%|██████████| 941814/941814 [00:01<00:00, 827803.49it/s]


In [ ]:
# 데이터 프레임 train_nli에서 컬럼 sentence1, sentence2 데이터를 순서대로 각각 추출한다 -> for ~
def data_convert( sen1s, sen2s, max_length, tokeni ):
  '''
    sen1s : 문장으로 구성된 말뭉치 1번세트
    sen2s : 문장으로 구성된 말뭉치 2번세트
    max_length : 토큰화 과정시 문장 최대길이 지정
    tokeni : 토큰화 도구(토크나이저)
  '''
  # 각각 데이터를 담을 그릇
  input_ids       = list() 
  token_type_ids  = list() 
  attention_masks = list() 

  for s1, s2 in tqdm(zip(sen1s, sen2s), total=len(sen1s) ):
    # 인코딩 처리
    results = tokeni.encode_plus( s1, s2, max_length=max_length, pad_to_multiple_of=True )
    # 각각 데이터들을 그릇에 추가한다
    input_ids.append(       results['input_ids'] )
    token_type_ids.append(  results['token_type_ids'] )
    attention_masks.append( results['attention_mask'] )
    pass
  
  return input_ids, token_type_ids, attention_masks

In [ ]:
X_train = data_convert( train_nli.sentence1, train_nli.sentence2, max_length, tokenizer )

  0%|          | 205/941814 [00:00<07:40, 2043.08it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 1780/941814 [00:01<12:50, 1219.96it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have b

In [ ]:
# 결과 형태 확인
len(X_train), type( X_train[0] ), type( X_train[1] ), type( X_train[2] )

(3, list, list, list)

In [ ]:
# 검증용, 테스트용 컨버전 모두 진행
X_val  = data_convert( val_xnli.sentence1,  val_xnli.sentence2,  max_length, tokenizer )
X_test = data_convert( test_xnli.sentence1, test_xnli.sentence2, max_length, tokenizer )

100%|██████████| 5010/5010 [00:03<00:00, 1610.39it/s]


In [ ]:
print( 'input_ids', X_test[0][0] )

input_ids [2, 8911, 16, 717, 2259, 3724, 2170, 9149, 3628, 2446, 2232, 1889, 2118, 1380, 2886, 3683, 16, 717, 2259, 3760, 8642, 2371, 2088, 16, 3983, 636, 2170, 2318, 3690, 3758, 2205, 2318, 859, 2359, 2062, 18, 3, 717, 2259, 636, 2522, 3690, 3758, 2205, 2118, 1380, 2886, 2062, 18, 3]


In [ ]:
print( 'token_type_ids', X_test[1][0] )

token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print( 'attention_mask', X_test[2][0] )

attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# 문장 복원
tokenizer.decode( X_test[0][0] )

'[CLS] 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다. [SEP] 나는 그와 다시 이야기하지 않았다. [SEP]'

# 정답 변환 처리

- 분류
  - 이진 분류
  - 다중 분류
    - label 값이 본연의 데이터를 사용
      - en, fr, id, tl, …
    - **label 값을 정수 변환 처리**
      - 0, 1 , 2, 3, …
- 컴파일 시 손실함수 선정할 때 정답의 형태에 따라 맞춰야한다
  - 컴파일 구성 시 확인

In [ ]:
train_nli.columns

Index(['sentence1', 'sentence2', 'gold_label'], dtype='object')

In [ ]:
train_labels = train_nli['gold_label'].tolist()
val_labels   = val_xnli['gold_label'].tolist()
test_labels  = test_xnli['gold_label'].tolist()

In [ ]:
# LabelEncoder : 정답을 정수로 변환 처리
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_enc = LabelEncoder()

In [ ]:
# 정답 대비 정수 값 세트 획득
label_enc.fit(train_labels)

LabelEncoder()

In [ ]:
# 변환 처리
y_train = label_enc.transform(train_labels)
y_val = label_enc.transform(val_labels)
y_test = label_enc.transform(test_labels)

In [ ]:
print(y_test)

[0 1 2 ... 2 0 1]


## 데이터 저장

- X_train, X_val, X_test
- y_train, y_val, y_test

In [ ]:
saved_path = '/content/drive/MyDrive/빅데이터_딥러닝/Day04/multi_clf_data/'
import pickle

In [ ]:
with open(saved_path+'data.dat', 'wb') as f:
  pickle.dump([X_train, X_val, X_test, y_train, y_val, y_test], f)

In [ ]:
# 로드하여 확인
with open(saved_path+'data.dat', 'rb') as f:
    data = pickle.load(f)

In [ ]:
print(len(data), len(data[0]))

6 3


In [ ]:
data[0][0]

[[2,
  21,
  2179,
  2048,
  19769,
  2015,
  2200,
  553,
  27135,
  19769,
  2333,
  2078,
  2069,
  1889,
  2259,
  4576,
  18,
  3,
  3997,
  2116,
  553,
  27135,
  6379,
  2069,
  6159,
  1513,
  2062,
  18,
  3],
 [2,
  732,
  2116,
  1041,
  2371,
  2471,
  2052,
  16,
  18459,
  2200,
  14517,
  2112,
  25190,
  2145,
  1,
  22987,
  2116,
  2170,
  16284,
  1564,
  2460,
  720,
  5110,
  2069,
  568,
  2275,
  18,
  3,
  18459,
  27135,
  4834,
  2259,
  13936,
  2170,
  4391,
  5643,
  5110,
  2069,
  568,
  2275,
  18,
  3],
 [2,
  15572,
  1116,
  16418,
  2069,
  5814,
  31302,
  1537,
  2170,
  30647,
  2185,
  5483,
  7775,
  5887,
  2073,
  5562,
  7014,
  7245,
  2205,
  2259,
  3935,
  31221,
  11142,
  12190,
  18,
  3,
  5562,
  4654,
  2073,
  3935,
  31221,
  3747,
  28674,
  18,
  3],
 [2,
  573,
  2073,
  10727,
  2138,
  1511,
  2073,
  4358,
  2778,
  9019,
  2116,
  13730,
  7285,
  6637,
  2259,
  3850,
  20758,
  11264,
  2343,
  1108,
  2942,
  2345,
  21

# 모델 구성 및 학습(18장 이동)

- <a href='https://colab.research.google.com/drive/1mzZtBg7DhYYYFbeSGd_LOCKj00UkP43U'>이동</a>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint